# Lomas Client Side: Using Smartnoise-SQL

This notebook showcases how researcher could use lomas platform with Smartnoise-SQL. It explains the different functionnalities provided by the `lomas-client` client library to interact with lomas server.

The secure data are never visible by researchers. They can only access to differentially private responses via queries to the server.

Each user has access to one or multiple projects and for each dataset has a limited budget $\epsilon$, $\delta$.

In this notebook the researcher is a penguin researcher named Dr. Antarctica. She aims to do a grounbdbreaking research on various penguins data.

## Step 1: Install the library
To interact with the secure server on which the data is stored, Dr.Antartica first needs to install the library `lomas-client` on her local developping environment. 

It can be installed via the pip command:

In [ ]:
# !pip install lomas_client

Or using a local version of the clien

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
from lomas_client.client import Client
import numpy as np

## Step 2: Initialise the client

Once the library is installed, a Client object must be created. It is responsible for sending sending requests to the server and processing responses in the local environment. It enables a seamless interaction with the server. 

To create the client, Dr. Antartica needs to give it a few parameters:
- a url: the root application endpoint to the remote secure server.
- user_name: her name as registered in the database (Dr. Alice Antartica)
- dataset_name: the name of the dataset that she wants to query (PENGUIN)

She will only be able to query on the real dataset if the administratir has previously made her an account in the database, given her access to the PENGUIN dataset and has given her some $\epsilon$, $\delta$ privacy loss budget.

In [3]:
APP_URL = "http://lomas_server"
USER_NAME = "Dr. Antartica"
DATASET_NAME = "PENGUIN"
client = Client(url=APP_URL, user_name = USER_NAME, dataset_name = DATASET_NAME)

And that's it for the preparation. She is now ready to use the various functionnalities offered by `lomas-client`.

## Step 3: Getting dataset metadata

In [6]:
metadata = client.get_dataset_metadata()
metadata

{'max_ids': 1,
 'row_privacy': True,
 'censor_dims': False,
 'columns': {'species': {'type': 'string',
   'cardinality': 3,
   'categories': ['Adelie', 'Chinstrap', 'Gentoo']},
  'island': {'type': 'string',
   'cardinality': 3,
   'categories': ['Torgersen', 'Biscoe', 'Dream']},
  'bill_length_mm': {'type': 'float', 'lower': 30.0, 'upper': 65.0},
  'bill_depth_mm': {'type': 'float', 'lower': 13.0, 'upper': 23.0},
  'flipper_length_mm': {'type': 'float', 'lower': 150.0, 'upper': 250.0},
  'body_mass_g': {'type': 'float', 'lower': 2000.0, 'upper': 7000.0},
  'sex': {'type': 'string',
   'cardinality': 2,
   'categories': ['MALE', 'FEMALE']}},
 'rows': 344}

In [48]:
nb_penguin = metadata['rows']
print(f"Number of penguins: {nb_penguin}.")

Number of penguins: 344.


In [50]:
columns = metadata["columns"].keys()
columns

dict_keys(['species', 'island', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex'])

## Step 4: Average bill length with Smartnoise-SQL

### Query dummy dataset

Now that she has an idea of what the data looks like, she wants to start querying the real dataset to for her research. However, before this other tools are at her disposal to reduce potential error risks and avoid spending budget on irrelevant queries. Of course, this does not have any impact on the budget.

It is possible to specify the flag `dummy=True` in the various queries to perform the query on the dummy dataset instead of the real dataset and ensure that the queries are doing what is expected of them. 

In [8]:
# Average bill length in mm
QUERY = "SELECT AVG(bill_length_mm) AS avg_bill_length_mm FROM df"

In [14]:
EPSILON = 1.0
DELTA = 1e-5

In [15]:
# On the remote server dummy dataframe
dummy_res = client.smartnoise_sql_query(
    query = QUERY,  
    epsilon = EPSILON,
    delta = DELTA,
    dummy = True,
)

In [16]:
avg_bl_dummy = np.round(dummy_res['query_response']["avg_bill_length_mm"][0], 2)
f"Average bill length on dummy: {avg_bl_dummy}mm."

'Average bill length on dummy: 46.84mm.'

### Estimate cost of a query
Dr. Antartica checks the budget that computing the average bill length will really cost her if she asks the query with an `epsilon` and a `delta`.

In [20]:
cost = client.estimate_smartnoise_sql_cost(
    query = QUERY, 
    epsilon = EPSILON, 
    delta = DELTA
)

In [21]:
f'This query would actually cost her {cost["epsilon_cost"]} epsilon and {cost["delta_cost"]} delta.'

'This query would actually cost her 2.0 epsilon and 5.000000000032756e-06 delta.'

This is actually the double than what she put in input. In the background, Smartnoise-SQL decomposes the DP query in multiple other queries and the budget given in input is spent on each of these sub-queries. Here for the average, we need a sum divided by a count, hence `EPSILON` is spent once for the sum and then once more for the count. (see NOTE below for tips and explanation).

### Query real dataset
Dr. Antartica is ready to query on the real dataset and get a differentially private response of the average bill length. By default, the flag `dummy` is False so setting it is optional. She uses the values of `epsilon` and `delta` that she selected just before.

Careful: This command DOES spend the budget of the user and the remaining budget is updated for every query.

In [28]:
avg_bill_length_response = client.smartnoise_sql_query(
    query = QUERY,  
    epsilon = EPSILON, 
    delta = DELTA,
    dummy = False
)

In [30]:
avg_bill_length = avg_bill_length_response['query_response']['avg_bill_length_mm'].iloc[0]
print(f"Average bill length on private data: {np.round(avg_bill_length, 2)}mm.")

Average bill length on private data: 44.11mm.


After each query on the real dataset, the budget informations are also returned to the researcher. It is possible possible to check the remaining budget again afterwards:

## Step 4: Penguin statistics

### Confidence intervals for flipper length over the whole population

She is first interested to have a better idea of the distribution of bill length of all species. She already has the number of penguins (=number of rows as `max_ids=1`) from the metadata and the average bill length from step 3, so she just need to compute the standard deviation. As it is just an exploration step, she uses very little budget values.

In [39]:
QUERY = "SELECT STD(bill_length_mm) AS std_bill_length_mm FROM df"

She again first verifies that her query works on the dummy dataset:

In [32]:
dummy_res = client.smartnoise_sql_query(
    query = QUERY, 
    epsilon = 1.0, 
    delta = 1e-5, 
    dummy = True
)

In [38]:
dummy_std = np.round(dummy_res['query_response']['std_bill_length_mm'].iloc[0], 2)
f"The dummy standard variation is {dummy_std}."

'The dummy standard variation is 2.78.'

The syntax of the query works, now she checks the budget:

In [41]:
cost = client.estimate_smartnoise_sql_cost(
    query = QUERY, 
    epsilon = 1.0, 
    delta = 1e-5
)

In [42]:
f'This query would actually cost her {cost["epsilon_cost"]} epsilon and {cost["delta_cost"]} delta.'

'This query would actually cost her 3.0 epsilon and 5.000000000032756e-06 delta.'

This times it is three times the budget as the standard deviation needs the average, then a difference and a count again. 

In [43]:
response = client.smartnoise_sql_query(
    query = QUERY,
    epsilon = 1.0,
    delta = 1e-5
)

In [51]:
std_bill_length = response['std_bill_length_mm'].iloc[0]
print(f"Standard deviation of bill length: {np.round(std_bill_length, 2)}.")

Standard deviation of bill length: 6.89.


She can now do all the postprocessing that she wants with the returned data without adding any privacy risk. 

In [52]:
# Get standard error
standard_error = std_bill_length/np.sqrt(nb_penguin)
print(f"Standard error of bill length: {np.round(standard_error, 2)}.")

Standard error of bill length: 0.37.


In [53]:
 # Compute the 95% confidence interval
ZSCORE = 1.96
lower_bound, upper_bound = avg_bill_length - ZSCORE*standard_error, avg_bill_length + ZSCORE*standard_error
print(f"The 95% confidence interval of the bill length of all penguins is [{np.round(lower_bound, 2)}, {np.round(upper_bound, 2)}].")

The 95% confidence interval of the bill length of all penguins is [43.38, 44.84].


## Note on budget with Smartnoise-SQL

In [55]:
epsilon = 1.0
delta = 1e-5

In [57]:
QUERY = "SELECT STD(bill_length_mm) AS std_bill_length_mm FROM df"
cost = client.estimate_smartnoise_sql_cost(query = QUERY, epsilon = epsilon, delta = delta)
cost

{'epsilon_cost': 3.0, 'delta_cost': 5.000000000032756e-06}

In [62]:
QUERY = "SELECT AVG(bill_length_mm) AS avg_bl, STD(bill_length_mm) as std_bl FROM df"
cost = client.estimate_smartnoise_sql_cost(query = QUERY, epsilon = epsilon, delta = delta)
cost

{'epsilon_cost': 3.0, 'delta_cost': 5.000000000032756e-06}

In [61]:
QUERY = "SELECT COUNT(bill_length_mm) AS count_bl, AVG(bill_length_mm) AS avg_bl, STD(bill_length_mm) as std_bl FROM df"
cost = client.estimate_smartnoise_sql_cost(query = QUERY, epsilon = epsilon, delta = delta)
cost

{'epsilon_cost': 3.0, 'delta_cost': 5.000000000032756e-06}

In [72]:
QUERY = "SELECT COUNT(bill_length_mm) AS count_bl, AVG(bill_length_mm) AS avg_bl, STD(bill_length_mm) as std_bl FROM df GROUP BY species"
cost = client.estimate_smartnoise_sql_cost(query = QUERY, epsilon = epsilon, delta = delta)
cost

{'epsilon_cost': 3.0, 'delta_cost': 5.000000000032756e-06}

### Hypothesis testing

So, Dr. Antartica has now the opportunity to study the various penguins dimensions and will do an hypotheses testing analysis to discover if flipper length differ between the penguins species.

She will do a two-tailed two-sample $t$ test.

- The null hypothese $H_0$ is flipper length does not differ between species.
- The alternative hypothesis $H_a$ is flipper length does differ between species.

She set the level of significance at 0.05.

In [63]:
CRITICAL_VALUE = 0.05

In [64]:
QUERY = "SELECT \
        species AS species, \
        COUNT(bill_length_mm) AS nb_penguin,  \
        AVG(bill_length_mm) AS avg_bill_length_mm, \
        STD(bill_length_mm) AS std_bill_length_mm \
        FROM df GROUP BY species"

She estimates how much budget it would really cost:

In [65]:
client.estimate_smartnoise_sql_cost(query = QUERY, epsilon = 1, delta = 1e-4)

{'epsilon_cost': 3.0, 'delta_cost': 4.999999999999449e-05}

The real cost seems to be 3 times the epsilon that she sets. It is a lot but she tries on the dummy dataset to verify all is working properly.

In [66]:
dummy_res = client.smartnoise_sql_query(query = QUERY, epsilon = 1, delta = 1.0, dummy = True)
dummy_res

Server error status 400: {"InvalidQueryException":"SQL Reader generated NAN results. Epsilon: 1.0 and Delta: 1.0 are too small to generate output."}


She did not give enough budget for the query to work. This is why there are 'NANs' in the output. She has to spend more budget for the query to work.

In [70]:
dummy_res = client.smartnoise_sql_query(query = QUERY, epsilon = 2.0, delta = 1e-4, dummy = True)
dummy_res

{'query_response':      species  nb_penguin  avg_bill_length_mm  std_bill_length_mm
 0     Adelie          38           46.293151           13.104888
 1  Chinstrap          33           48.744280           14.518132
 2     Gentoo          28           43.582573           11.503930}

If it errors, she might need to re-run the query a few times until it works. The budget is not affected by dummy queries anyway.

In [71]:
flipper_length_response = client.smartnoise_sql_query(query = QUERY, epsilon = 2.0, delta = 1e-4)

Server error status 400: {"InvalidQueryException":"Not enough budget for this query epsilon remaining 5.0, delta remaining 0.004989999999999935."}


And now she should not have any remaining budget:

But she can do her post-processing and hypothesis analysis.

In [46]:
df_flipper = flipper_length_response['query_response']
df_flipper

,species,nb_penguin,avg_bill_length_mm,std_bill_length_mm
0,Adelie,150,38.649887,3.997587
1,Chinstrap,67,49.285002,5.859511
2,Gentoo,122,47.557167,4.643492


And finally the $t$-test:

In [47]:
def t_test(avg_1, avg_2, std_1, std_2, nb_1, nb_2):
    standard_error = (std_1 * (nb_1 - 1) + std_2 * (nb_2 - 1))/(nb_1 + nb_2 - 2)
    return (avg_1 - avg_2)/np.sqrt(standard_error**2*(1/nb_1 + 1 /nb_2))

In [48]:
nb_0, avg_0, std_0 = df_flipper[['nb_penguin', 'avg_bill_length_mm', 'std_bill_length_mm']].iloc[0]
nb_1, avg_1, std_1 = df_flipper[['nb_penguin', 'avg_bill_length_mm', 'std_bill_length_mm']].iloc[1]
nb_2, avg_2, std_2 = df_flipper[['nb_penguin', 'avg_bill_length_mm', 'std_bill_length_mm']].iloc[2]

In [49]:
t_01 = t_test(avg_0, avg_1, std_0, std_1, nb_0, nb_1)
t_02 = t_test(avg_0, avg_2, std_0, std_2, nb_0, nb_2)
t_12 = t_test(avg_1, avg_2, std_1, std_2, nb_1, nb_2)

print(f"T test between specie 0 and specie 1: {np.round(t_01, 2)}.  Reject null hypothesis: {abs(t_01) > CRITICAL_VALUE}.")
print(f"T test between specie 0 and specie 2: {np.round(t_02, 2)}. Reject null hypothesis: {abs(t_02) > CRITICAL_VALUE}.")
print(f"T test between specie 1 and specie 2: {np.round(t_12, 2)}.   Reject null hypothesis: {abs(t_12) > CRITICAL_VALUE}.")

T test between specie 0 and specie 1: -15.84.  Reject null hypothesis: True.
T test between specie 0 and specie 2: -17.04. Reject null hypothesis: True.
T test between specie 1 and specie 2: 2.24.   Reject null hypothesis: True.


All t-tests are above the critical value of 0.5. She can reject the null hypothesis.

She finally computes the confidence intervals for the flipper length of each species

In [50]:
ZSCORE = 1.96
df_flipper['standard_error'] = df_flipper['std_bill_length_mm']/np.sqrt(df_flipper['nb_penguin'])
df_flipper['ci_95_lower_bound'] = df_flipper['avg_bill_length_mm'] - ZSCORE * df_flipper['standard_error']
df_flipper['ci_95_upper_bound'] = df_flipper['avg_bill_length_mm'] + ZSCORE * df_flipper['standard_error']
df_flipper

,species,nb_penguin,avg_bill_length_mm,std_bill_length_mm,standard_error,ci_95_lower_bound,ci_95_upper_bound
0,Adelie,150,38.649887,3.997587,0.326402,38.010140,39.289634
1,Chinstrap,67,49.285002,5.859511,0.715853,47.881930,50.688074
2,Gentoo,122,47.557167,4.643492,0.420402,46.733179,48.381155
